In [1]:
import numpy as np
import pandas as pd
# importing libraries
import statsmodels.api as sm

In [2]:
patient_summary =  pd.read_csv("data/patient_summary.csv")
treatment = pd.read_csv("data/admit_treatment.csv")

print(patient_summary.columns,treatment.columns)

/home/erichschulman/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (9,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/erichschulman/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Index(['pat_id', 'state', 'age_range', 'race', 'gender', 'ethnicity', 'mcstat',
       'cvrg_stat', 'primdiag', 'dofd', 'dod', 'primcause', 'netnumb'],
      dtype='object') Index(['pat_id', 'prov_nbr', 'Admit_Date', 'Admit_Reason', 'Discharge_Date',
       'Discharge_Reason', 'Treatment_Start_Date', 'Dialysis_Setting',
       'Treatment_Type', 'Sessions_per_Week', 'Dialysis_Type',
       'Transplant_Type', 'Transplant_Date', 'Modality'],
      dtype='object')


In [3]:
#TODO could expand to multiple years...
#TODO this number seems low?
year = 2014
patient_summary_small = patient_summary.copy()
#patient_summary_small['dofd'] = patient_summary_small['dofd'].astype(int,errors='ignore')

patient_summary_small = patient_summary_small[patient_summary_small['dofd'].apply(lambda x: str(x).isdigit()) ]
patient_summary_small['dofd'] = patient_summary_small['dofd'].astype(int,errors='ignore')
patient_summary_small = patient_summary_small[ (patient_summary_small['dofd']>=year) ]
print(patient_summary_small.shape)

(535337, 13)


In [4]:
provider_patient = pd.merge(patient_summary_small,treatment[ (treatment['Treatment_Type']=='DIALYSIS')
                                                            & (treatment['Dialysis_Setting']!='') ],
                            on=['pat_id'], how='inner')

In [5]:
print(provider_patient.shape)

(1181342, 26)


In [6]:
dummies  = pd.get_dummies( provider_patient[['cvrg_stat','Dialysis_Setting']])
dummies[['pat_id','state']] = provider_patient[['pat_id','state']]

coverage_columns = []
dialysis_columns = []
for col in list(dummies.columns):
    if 'cvrg_stat' in col:
        coverage_columns.append(col)
    if 'Dialysis_Setting' in col:
        dialysis_columns.append(col)

print(coverage_columns,dialysis_columns)

summarize_dummies = dummies.groupby(['pat_id','state'],as_index=False).mean()
#summarize_dummies['cvrg_switchers'] = summarize_dummies[coverage_columns].sum(axis=1) > 1
#summarize_dummies['dialysis_switchers'] = summarize_dummies[dialysis_columns].sum(axis=1) > 1

state_dummies = summarize_dummies.groupby('state').mean()
state_dummies = state_dummies.drop(columns='pat_id')

state_dummies['count'] = summarize_dummies.groupby('state').count()['pat_id']
state_dummies.to_csv('results_%s.csv'%year)

print(summarize_dummies.shape)
print()

['cvrg_stat_ ', 'cvrg_stat_Currently enrolled in Medicare Coverage', 'cvrg_stat_Medicare application pending', 'cvrg_stat_No Medicare coverage'] ['Dialysis_Setting_Dialysis Facility/Center', 'Dialysis_Setting_Home', 'Dialysis_Setting_SNF/Long Term Care Facility']
(535318, 9)



In [7]:
print(state_dummies[['cvrg_stat_No Medicare coverage','count']][state_dummies['cvrg_stat_No Medicare coverage'].max() 
                    == state_dummies['cvrg_stat_No Medicare coverage']])

print(state_dummies.std())
print('---')
print(state_dummies.max())

       cvrg_stat_No Medicare coverage  count
state                                       
Other                        0.459474   7193
cvrg_stat_                                               0.021622
cvrg_stat_Currently enrolled in Medicare Coverage        0.085656
cvrg_stat_Medicare application pending                   0.005379
cvrg_stat_No Medicare coverage                           0.079521
Dialysis_Setting_Dialysis Facility/Center                0.044439
Dialysis_Setting_Home                                    0.042464
Dialysis_Setting_SNF/Long Term Care Facility             0.013712
count                                                13467.227631
dtype: float64
---
cvrg_stat_                                               0.111111
cvrg_stat_Currently enrolled in Medicare Coverage        0.858647
cvrg_stat_Medicare application pending                   0.026786
cvrg_stat_No Medicare coverage                           0.459474
Dialysis_Setting_Dialysis Facility/Center             

In [8]:
x_name = list(summarize_dummies[dialysis_columns].columns)
y_name = ['cvrg_stat_Currently enrolled in Medicare Coverage']
# building the model and fitting the data
sm.Logit(summarize_dummies[y_name], (summarize_dummies[x_name]) ).fit().summary()

Optimization terminated successfully.
         Current function value: 0.669761
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                                           Logit Regression Results                                          
=============================================================================================================
Dep. Variable:     cvrg_stat_Currently enrolled in Medicare Coverage   No. Observations:               535318
Model:                                                         Logit   Df Residuals:                   535315
Method:                                                          MLE   Df Model:                            2
Date:                                               Mon, 17 Oct 2022   Pseudo R-squ.:               0.0003470
Time:                                                       11:07:51   Log-Likelihood:            -3.5853e+05
converged:                                                      True   LL-Null:                   -3.5866e+05
Covariance Type:                                           nonrobust   LLR p-value:                 9.037e-55
================================================================================================================
                                                   coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Dialysis_Setting_Dialysis Facility/Center        0.4184      0.003    136.758      0.000       0.412       0.424
Dialysis_Setting_Home                            0.5316      0.009     59.696      0.000       0.514       0.549
Dialysis_Setting_SNF/Long Term Care Facility     0.8626      0.041     20.881      0.000       0.782       0.944
================================================================================================================
"""

In [9]:
x_name = list(summarize_dummies[dialysis_columns].columns)
y_name = ['cvrg_stat_No Medicare coverage']
# building the model and fitting the data
sm.Logit(summarize_dummies[y_name], (summarize_dummies[x_name]) ).fit().summary()

Optimization terminated successfully.
         Current function value: 0.640263
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Logit Regression Results                                 
==========================================================================================
Dep. Variable:     cvrg_stat_No Medicare coverage   No. Observations:               535318
Model:                                      Logit   Df Residuals:                   535315
Method:                                       MLE   Df Model:                            2
Date:                            Mon, 17 Oct 2022   Pseudo R-squ.:               0.0004321
Time:                                    11:07:52   Log-Likelihood:            -3.4274e+05
converged:                                   True   LL-Null:                   -3.4289e+05
Covariance Type:                        nonrobust   LLR p-value:                 4.512e-65
================================================================================================================
                                                   coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Dialysis_Setting_Dialysis Facility/Center       -0.6458      0.003   -204.908      0.000      -0.652      -0.640
Dialysis_Setting_Home                           -0.7923      0.009    -85.481      0.000      -0.810      -0.774
Dialysis_Setting_SNF/Long Term Care Facility    -1.0472      0.043    -24.361      0.000      -1.131      -0.963
================================================================================================================
"""

In [10]:
x_name = list(summarize_dummies[dialysis_columns].columns)
state_dummies = pd.get_dummies(summarize_dummies['state'])
summarize_dummies2 = summarize_dummies.copy()
state_names = list(state_dummies.columns)
print(state_names)
summarize_dummies2[state_names] = state_dummies[state_names]
y_name = ['cvrg_stat_Currently enrolled in Medicare Coverage']
# building the model and fitting the data
sm.Logit(summarize_dummies[y_name], summarize_dummies2[x_name+state_names]).fit().summary()

['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'Other', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
         Current function value: 0.663614
         Iterations: 35


/home/erichschulman/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                           Logit Regression Results                                          
=============================================================================================================
Dep. Variable:     cvrg_stat_Currently enrolled in Medicare Coverage   No. Observations:               535318
Model:                                                         Logit   Df Residuals:                   535263
Method:                                                          MLE   Df Model:                           54
Date:                                               Mon, 17 Oct 2022   Pseudo R-squ.:                0.009520
Time:                                                       11:08:04   Log-Likelihood:            -3.5524e+05
converged:                                                     False   LL-Null:                   -3.5866e+05
Covariance Type:                                           nonrobust   LLR p-value:                     0.000
================================================================================================================
                                                   coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Dialysis_Setting_Dialysis Facility/Center        0.5830        nan        nan        nan         nan         nan
Dialysis_Setting_Home                            0.6836        nan        nan        nan         nan         nan
Dialysis_Setting_SNF/Long Term Care Facility     1.1140        nan        nan        nan         nan         nan
AK                                               0.3966        nan        nan        nan         nan         nan
AL                                              -0.1565        nan        nan        nan         nan         nan
AR                                               0.2847        nan        nan        nan         nan         nan
AZ                                              -0.6117        nan        nan        nan         nan         nan
CA                                              -0.4104        nan        nan        nan         nan         nan
CO                                              -0.2088        nan        nan        nan         nan         nan
CT                                               0.1639        nan        nan        nan         nan         nan
DC                                              -0.2349        nan        nan        nan         nan         nan
DE                                              -0.0036        nan        nan        nan         nan         nan
FL                                              -0.1787        nan        nan        nan         nan         nan
GA                                              -0.0161        nan        nan        nan         nan         nan
HI                                              -0.1024        nan        nan        nan         nan         nan
IA                                               0.4911        nan        nan        nan         nan         nan
ID                                               0.6402        nan        nan        nan         nan         nan
IL                                              -0.2843        nan        nan        nan         nan         nan
IN                                              -0.1407        nan        nan        nan         nan         nan
KS                                               0.2374        nan        nan        nan         nan         nan
KY                                              -0.2133        nan        nan        nan         nan         nan
LA                                              -0.3235        nan        nan        nan         nan         nan
MA                                              -0.0453        nan        nan        nan         nan         nan
MD                      

In [11]:
#can use this to geocode?
#https://geocoding.geo.census.gov/geocoder/geographies/addressbatch?form


In [12]:
#demeaned approach

def demean(y_name,x_name,data=None,absorb=None,cluster=None): 

    y,X = data[ y_name], data[ x_name ]
    
    y_dot = y.copy()
    X_dot = X.copy()
    
    ybar = y.mean()
    Xbar = X.mean()

    
    for effect in absorb:
        y_dot = y_dot - y.groupby(data[effect]).transform('mean')
        X_dot = X_dot - X.groupby(data[effect]).transform('mean') 
    
    y_dot = y_dot + ybar
    X_dot = X_dot + Xbar
    return y_dot, X_dot

y_dot,X_dot = demean(y_name,x_name,data=summarize_dummies,absorb='state')

# building the model and fitting the data
sm.OLS(y_dot,X_dot).fit().summary()

KeyError: 's'